In [ ]:
# https://www.analyticsvidhya.com/blog/2018/11/implementation-faster-r-cnn-python-object-detection/

In [1]:
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile


In [2]:
pwd

'/home/scar3crow/Dropbox/WorkStation-Subrata/python'

In [12]:
%pylab inline

import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile

annotations = glob('/home/scar3crow/Downloads/BCCD_Dataset-master/BCCD/Annotations/*.xml')

df = []
cnt = 0
for file in annotations:
    prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
    filename = str(cnt) + '.jpg'
    row = []
    parsedXML = ET.parse(file)
    for node in parsedXML.getroot().iter('object'):
        blood_cells = node.find('name').text
        xmin = int(node.find('bndbox/xmin').text)
        xmax = int(node.find('bndbox/xmax').text)
        ymin = int(node.find('bndbox/ymin').text)
        ymax = int(node.find('bndbox/ymax').text)

        row = [prev_filename, filename, blood_cells, xmin, xmax,
        ymin, ymax]
    df.append(row)
    cnt += 1

data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'cell_type',
'xmin', 'xmax', 'ymin', 'ymax'])

data[['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('blood_cell_detection.csv', index=False)




Populating the interactive namespace from numpy and matplotlib


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [41]:
data.head()

,prev_filename,filename,cell_type,xmin,xmax,ymin,ymax
0,BloodImage_00111.jpg,0.jpg,Platelets,360,395,85,115
1,BloodImage_00057.jpg,1.jpg,RBC,410,503,290,390
2,BloodImage_00371.jpg,2.jpg,WBC,175,389,87,298
3,BloodImage_00020.jpg,3.jpg,Platelets,80,115,128,161
4,BloodImage_00178.jpg,4.jpg,RBC,122,237,319,413


In [42]:
len(data)

364

In [43]:
all_data_blood = pd.read_csv('test.csv')
print('Total no. of images = ', all_data_blood['filename'].nunique())
print('Total no. of data = ', len(all_data_blood))


FileNotFoundError: File b'test.csv' does not exist

In [7]:
all_data_blood.head(3)

,filename,cell_type,xmin,xmax,ymin,ymax
0,BloodImage_00000.jpg,WBC,260,491,177,376
1,BloodImage_00000.jpg,RBC,78,184,336,435
2,BloodImage_00000.jpg,RBC,63,169,237,336


In [8]:
all_data_blood.columns = ['previous', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']
all_data_blood.head(3)

,previous,cell_type,xmin,xmax,ymin,ymax
0,BloodImage_00000.jpg,WBC,260,491,177,376
1,BloodImage_00000.jpg,RBC,78,184,336,435
2,BloodImage_00000.jpg,RBC,63,169,237,336


In [9]:
col_list = ['0']*len(all_data_blood)

In [10]:
for j in range(len(data)):
    
    for i in range(len(all_data_blood)):
    
        if all_data_blood['previous'][i] == data['prev_filename'][j]:
            col_list[i] = data['filename'][j]


In [11]:
all_data_blood.insert(1, "image_names", col_list, True)

In [12]:
all_data_blood.head(3)

,previous,image_names,cell_type,xmin,xmax,ymin,ymax
0,BloodImage_00000.jpg,272.jpg,WBC,260,491,177,376
1,BloodImage_00000.jpg,272.jpg,RBC,78,184,336,435
2,BloodImage_00000.jpg,272.jpg,RBC,63,169,237,336


In [13]:
print('Total no. of images = ', all_data_blood['image_names'].nunique())
print('Total no. of data = ', len(all_data_blood))

Total no. of images =  364
Total no. of data =  4888


In [14]:
train = all_data_blood[0:3406].reset_index()
test = all_data_blood[3407:].reset_index()
print('Total no. of train images = ', train['image_names'].nunique())
print('Total no. of test images = ', test['image_names'].nunique())

Total no. of train images =  254
Total no. of test images =  110


In [15]:
len(train)

3406

In [16]:
test.head()

,index,previous,image_names,cell_type,xmin,xmax,ymin,ymax
0,3407,BloodImage_00288.jpg,170.jpg,RBC,67,163,359,452
1,3408,BloodImage_00288.jpg,170.jpg,RBC,339,435,387,480
2,3409,BloodImage_00288.jpg,170.jpg,RBC,407,520,328,422
3,3410,BloodImage_00288.jpg,170.jpg,RBC,444,544,229,321
4,3411,BloodImage_00288.jpg,170.jpg,RBC,420,522,126,223


In [17]:
train[['image_names', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('train_blood.csv', index = False)

In [18]:
import pandas as pd

In [19]:
exp_data = pd.DataFrame()
exp_data['format'] = train['image_names']
exp_data.shape

(3406, 1)

In [20]:
exp_data['format'][0]

'272.jpg'

In [21]:
exp_data['format'][0] = 'train_images/' + exp_data['format'][0]

In [22]:
exp_data['format'][9]

'272.jpg'

In [23]:
exp_data = pd.DataFrame()
exp_data['format'] = train['image_names']

# as the images are in train_images folder, add train_images before the image name
for i in range(exp_data.shape[0]):
    exp_data['format'][i] = 'train_images/' + exp_data['format'][i]

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(exp_data.shape[0]):
    exp_data['format'][i] = exp_data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['cell_type'][i]

exp_data.to_csv('train.txt', header=None, index=None, sep=' ')

In [24]:
exp_data

,format
0,"train_images/272.jpg,260,177,491,376,WBC"
1,"train_images/272.jpg,78,336,184,435,RBC"
2,"train_images/272.jpg,63,237,169,336,RBC"
3,"train_images/272.jpg,214,362,320,461,RBC"
4,"train_images/272.jpg,414,352,506,445,RBC"
5,"train_images/272.jpg,555,356,640,455,RBC"
6,"train_images/272.jpg,469,412,567,480,RBC"
7,"train_images/272.jpg,1,333,87,437,RBC"
8,"train_images/272.jpg,4,406,95,480,RBC"
9,"train_images/272.jpg,155,74,247,174,RBC"


In [25]:
cd keras-frcnn

/home/scar3crow/Dropbox/WorkStation-Subrata/python/keras-frcnn


In [27]:
train_frcnn.py -o simple -p train.txt

SyntaxError: invalid syntax (<ipython-input-27-8b374bb75208>, line 1)

In [28]:
ls

BCCD_Dataset-master.zip  measure_map.py    test.csv        Untitled1.ipynb
keras_frcnn/             README.md         test_frcnn.py   Untitled.ipynb
LICENSE                  requirements.txt  train_frcnn.py


In [29]:
cd rcnn

[Errno 2] No such file or directory: 'rcnn'
/home/scar3crow/Dropbox/WorkStation-Subrata/python/keras-frcnn


In [30]:
cd /home/scar3crow/Dropbox/WorkStation-Subrata/python

/home/scar3crow/Dropbox/WorkStation-Subrata/python


In [35]:
cd rcnn

/home/scar3crow/Dropbox/WorkStation-Subrata/python/rcnn


In [36]:
ls


convertor.py  keras-frcnn/  test_images/  train_blood.csv  train_images/


In [37]:
train_frcnn.py -o simple -p train.txt

SyntaxError: invalid syntax (<ipython-input-37-8b374bb75208>, line 1)